# PDI Analysis

This script investigates the output of our live stream detections in order to gain useful insights.
The goal is to create a "Physical Distancing Index" (`PDI`) that shows the trends and changes at
a camera feed while removing noise and perturbations from unwanted effects.

In this directory is a CSV file containing historical data for 7 cams.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [ ]:
data = pd.read_csv("counts.csv")

data.tail(10)

In [ ]:
data_dict = {
    stream_name: {
        "timestamp": data[data["stream_name"]==stream_name]["timestamp"],
        "count": data[data["stream_name"]==stream_name]["count"]
    } for stream_name in set(data["stream_name"])
}

print(list(data_dict.keys()))

In [ ]:
plt.figure(figsize=(20, 10))

for stream_name, d in data_dict.items():
    timestamps, counts = d["timestamp"], d["count"]
    datetimes = pd.to_datetime(timestamps, unit='s')
    
    plt.scatter(datetimes, counts, label=stream_name)
plt.xlabel("data")
plt.ylabel("number of detected objects")
plt.xlim([pd.Timestamp('2020-02-01'), pd.Timestamp('2020-04-03')])
plt.ylim([0, 100])
plt.legend();